In [3]:
# %load_ext IPython.extensions.autoreload
%reload_ext IPython.extensions.autoreload

In [4]:
import numpy as np
import pandas as pd

import openpyxl

import sys
import os
import runpy

import requests
from io import StringIO
import json

import json
import glob
from pathlib import Path

In [5]:
os.chdir('../..')
os.getcwd()


'c:\\Users\\jarem\\OneDrive - London School of Economics\\YEAR 2\\PP4V8 - policy paper\\policy-paper-repo'

# Operationalisation of variables

## Dane GUS

### Programme period: 2014 - 2020

In [6]:
PL_projects_14_20= pd.read_excel(r"data\inputs\dane_gus\lista_projektow_UE_2014_20\Lista_projektow_FE_2014_2020_02112025.xlsx", header = 2)

#### Clean up data

In [7]:
from src.functions import *

In [8]:
PL_projects_14_20.drop(columns=[
    "Forma finansowania/ Form of finance",
    "Projekt realizowany w ramach terytorialnych mechanizmów wdrażania/ Project implemented under territorial delivery mechanisms",
    "Działanie/ Measure",
    "Poddziałanie/ Submeasure",
    "Dziedzina działalności gospodarczej, której dotyczy projekt/ Area of economic activity",
    "Obszar wsparcia projektu/ Area of project intervention",
    "Cel projektu/ Project thematic objective",
    "Cel uzupełniający dla projektów EFS/ ESF secondary theme"
], inplace=True)
PL_projects_14_20 = clean_english_colnames(PL_projects_14_20, normalize='keep')   
PL_projects_14_20 = PL_projects_14_20[PL_projects_14_20['Fund'] != 'BAR'] 


In [9]:
PL_projects_14_20['Amount of EU co-financing (EUR)'] = PL_projects_14_20['Amount of EU co-financing (PLN, for ETC projects EUR)']/4.4975 # this exchange rate was taken from the report

In [10]:
# PL_projects_14_20.columns

In [11]:
df_locations = unnest_locations(PL_projects_14_20, 'Project location', 'Amount of EU co-financing (PLN, for ETC projects EUR)','Amount of EU co-financing (EUR)')

print(f"Original rows: {len(PL_projects_14_20)}")
print(f"Rows after unnesting: {len(df_locations)}")


Original rows: 103189
Rows after unnesting: 143327


In [12]:
# add country marker
df_locations = add_iso3_column(df_locations)

In [13]:
# display(df_locations[['Contract number','Project location', 'woj', 'pow', 'funding_split']].head(10))
# display(df_locations.head(3))

In [14]:
# save output
df_locations.to_csv(r"data\outputs\eu_flows\pol_1420_projects_by_location.csv", index=False)

In [15]:
# df_locations["Project location"].unique().tolist()

#### Reshape df so that powiats are row indexes

panel structure with treatment timing

In [16]:
df_locations = df_locations[df_locations['iso3'] == 'POL']

In [17]:
# Convert dates to datetime
df_locations['Project start date'] = pd.to_datetime(df_locations['Project start date'], errors='coerce')
df_locations['Project end date'] = pd.to_datetime(df_locations['Project end date'], errors='coerce')

# Extract year of first project (treatment year)
df_locations['year_start'] = df_locations['Project start date'].dt.year

# Create treatment variable: year when powiat first received EU funding
df_treatment_timing = df_locations.groupby('pow').agg({
    'year_start': 'min',  # first year treated
    'funding_split': 'sum',
    'Contract number': 'nunique'
}).reset_index()
df_treatment_timing.columns = ['pow', 'treatment_year', 'total_funding', 'num_projects']

# Create binary treatment indicator for each year
# For staggered DiD: treated = 1 if year >= treatment_year for that powiat
years = range(df_locations['year_start'].min(), df_locations['year_start'].max() + 1)
treatment_data = []

for pow in df_treatment_timing['pow'].unique():
    treat_year = df_treatment_timing[df_treatment_timing['pow'] == pow]['treatment_year'].values[0]
    for year in years:
        treatment_data.append({
            'pow': pow,
            'year': year,
            'treated': 1 if year >= treat_year else 0,
            'time_to_treatment': year - treat_year if year >= treat_year else None
        })

df_staggered_did = pd.DataFrame(treatment_data)

# Add cumulative funding received by that year (for intensity)
df_funding_by_year = df_locations.groupby(['pow', 'year_start'])['funding_split'].sum().reset_index()
df_funding_by_year.columns = ['pow', 'year', 'funding_that_year']
df_funding_by_year['funding_cum'] = df_funding_by_year.groupby('pow')['funding_that_year'].cumsum()

df_staggered_did = df_staggered_did.merge(df_funding_by_year[['pow', 'year', 'funding_cum']], 
                                            on=['pow', 'year'], how='left')
df_staggered_did['funding_cum'] = df_staggered_did['funding_cum'].fillna(0)



In [18]:
print("Staggered DiD treatment structure:")
display(df_staggered_did.head(10))

# Summary: which powiats treated when?
display(df_treatment_timing.sort_values('treatment_year').head(10))

Staggered DiD treatment structure:


,pow,year,treated,time_to_treatment,funding_cum
0,Biała Podlaska,2014,1,0.0,2.646463e+07
1,Biała Podlaska,2015,1,1.0,3.447804e+07
2,Biała Podlaska,2016,1,2.0,1.091886e+08
3,Biała Podlaska,2017,1,3.0,1.469402e+08
4,Biała Podlaska,2018,1,4.0,2.116581e+08
5,Biała Podlaska,2019,1,5.0,2.339718e+08
6,Biała Podlaska,2020,1,6.0,2.987195e+08
7,Biała Podlaska,2021,1,7.0,3.161128e+08
8,Biała Podlaska,2022,1,8.0,3.198439e+08
9,Biała Podlaska,2023,1,9.0,3.219366e+08


,pow,treatment_year,total_funding,num_projects
394,żuromiński,2014,1.730454e+08,154
369,Łódź,2014,7.612179e+09,2551
370,łaski,2014,1.905702e+08,202
371,łańcucki,2014,4.494919e+08,260
372,łobeski,2014,2.266230e+08,134
373,łomżyński,2014,4.534121e+08,330
27,Klaipedos rajono savivaldybe,2014,2.267455e+06,23
375,łowicki,2014,3.934279e+08,301
376,łukowski,2014,5.533870e+08,516
377,łódzki wschodni,2014,4.552300e+08,534


In [19]:
df_treatment_timing.treatment_year.value_counts().sort_index()

treatment_year
2014    360
2015     27
2016      8
2017      2
2022      1
Name: count, dtype: int64

In [20]:
# Save
df_staggered_did.to_parquet(r"data\outputs\eu_flows\pol_1420_treatment_staggered_did.parquet", index=False)
df_treatment_timing.to_parquet(r"data\outputs\eu_flows\pol_1420_treatment_timing.parquet", index=False)

### Programme period 2021-2027

In [21]:
PL_projects_21_27 = pd.read_excel(r"data\inputs\dane_gus\lista_projektow_UE_2021_27\Lista_projektow_FE_2021_2027_02112025.xlsx", header = 1)

#### Clean up 

In [22]:
from src.functions import *

In [23]:
PL_projects_21_27.drop(columns=[
    "Działanie/ Measure",
    # "Cel szczegółowy/ Specific objective",
    "Priorytet/ Priority", 
    "Działanie/ Measure",
    "Numer identyfikacyjny statku w rejestrze floty rybackiej UE/ The Union fishing fleet register identification number",
    "Nazwa wykonawcy kontraktu/ The contractor’s name",
    "Kategoria wsparcia/ Type of intervention"
], inplace=True)

PL_projects_21_27 = clean_english_colnames(PL_projects_21_27, normalize='keep')   
PL_projects_21_27 = PL_projects_21_27[PL_projects_21_27['Fund'] != 'BAR'] 

In [24]:
PL_projects_21_27["EU co-financing (EUR)"] = PL_projects_21_27["EU co-financing (PLN)"]/4.4975 # this exchange rate was taken from the report

In [25]:
# PL_projects_21_27.head()

In [26]:
PL_projects_21_27.columns

Index(['Project name', 'Project summary', 'Contract number',
       'Beneficiary name', 'Fund', 'Specific objective', 'Programme',
       'Total project value (PLN)', 'Union co-financing rate (%)',
       'EU co-financing (PLN)', 'EURO exchange rate', 'Project location',
       'Project start date', 'Project end date', 'EU co-financing (EUR)'],
      dtype='object')

In [27]:
df_locations = unnest_locations_with_gmina(PL_projects_21_27, 'Project location', 'EU co-financing (PLN)','EU co-financing (EUR)')

print(f"Original rows: {len(PL_projects_21_27)}")
print(f"Rows after unnesting: {len(df_locations)}")

Original rows: 23826
Rows after unnesting: 50011


In [28]:
# save output
df_locations.to_csv(r"data\outputs\eu_flows\pol_21_27_projects_by_location.csv", index=False)

In [29]:
df_locations = add_iso3_column(df_locations)
# df_locations = df_locations[df_locations['iso3'] == 'POL']


In [30]:
# df_locations["pow"].unique()

In [31]:
df_locations.head(3)

,Project name,Project summary,Contract number,Beneficiary name,Fund,Specific objective,Programme,Total project value (PLN),Union co-financing rate (%),EU co-financing (PLN),EURO exchange rate,Project location,Project start date,Project end date,EU co-financing (EUR),woj,pow,gmina,funding_split,iso3
0,Poprawa warunków recepcyjnych dla osób ubiegaj...,Polski system recepcyjny musi być przygotowany...,FAMI.01.01-IZ.00-0001/24,Szef Urzędu do Spraw Cudzoziemców,FAMI,FAMI.1 Wzmacnianie i rozwijanie wszystkich asp...,"Fundusz Azylu, Migracji i Integracji",41273504.53,80.0,33018803.62,NaN,"WOJ: LUBELSKIE, POW: Biała Podlaska, GM: Biała...",2024-10-01 23:59:59,2028-12-31 23:59:59,7.341591e+06,LUBELSKIE,Biała Podlaska,Biała Podlaska,16509401.81,POL
1,Poprawa warunków recepcyjnych dla osób ubiegaj...,Polski system recepcyjny musi być przygotowany...,FAMI.01.01-IZ.00-0001/24,Szef Urzędu do Spraw Cudzoziemców,FAMI,FAMI.1 Wzmacnianie i rozwijanie wszystkich asp...,"Fundusz Azylu, Migracji i Integracji",41273504.53,80.0,33018803.62,NaN,"WOJ: PODLASKIE, POW: zambrowski, GM: Zambrów",2024-10-01 23:59:59,2028-12-31 23:59:59,7.341591e+06,PODLASKIE,zambrowski,Zambrów,16509401.81,POL
2,Bezpieczna Przystań - Centrum Pomocy Kobietom ...,Szef Urzędu do Spraw Cudzoziemców (UdSC) jest ...,FAMI.01.01-IZ.00-0001/25,Szef Urzędu do Spraw Cudzoziemców,FAMI,FAMI.1 Wzmacnianie i rozwijanie wszystkich asp...,"Fundusz Azylu, Migracji i Integracji",67327046.73,75.0,50495285.04,NaN,"WOJ: MAZOWIECKIE, POW: legionowski, GM: Serock",2025-05-01 23:59:59,2029-12-31 23:59:59,1.122741e+07,MAZOWIECKIE,legionowski,Serock,50495285.04,POL


#### Reshape df so that powiats are row indexes

panel structure with treatment timing

In [32]:
# Convert dates to datetime
df_locations['Project start date'] = pd.to_datetime(df_locations['Project start date'], errors='coerce')
df_locations['Project end date'] = pd.to_datetime(df_locations['Project end date'], errors='coerce')

# Extract year of first project (treatment year)
df_locations['year_start'] = df_locations['Project start date'].dt.year

# --- Powiat-level treatment timing (kept for reference) ---
df_treatment_timing = df_locations.groupby('pow').agg({
    'year_start': 'min',    # first year treated
    'funding_split': 'sum',
    'Contract number': 'nunique'
}).reset_index()
df_treatment_timing.columns = ['pow', 'treatment_year', 'total_funding', 'num_projects']

# --- Gmina-level treatment timing ---
# Group by pow + gmina (gmina may be None for some rows)
df_treatment_timing_gmina = df_locations.groupby(['pow', 'gmina']).agg({
    'year_start': 'min',
    'funding_split': 'sum',
    'Contract number': 'nunique'
}).reset_index()
df_treatment_timing_gmina.columns = ['pow', 'gmina', 'treatment_year', 'total_funding', 'num_projects']

# Create binary treatment indicator for each year at gmina level
years = range(int(df_locations['year_start'].min()), int(df_locations['year_start'].max()) + 1)
treatment_data = []
for _, row in df_treatment_timing_gmina.iterrows():
    pow_ = row['pow']
    gmina_ = row['gmina']
    treat_year = int(row['treatment_year']) if pd.notna(row['treatment_year']) else None
    for year in years:
        treated = 1 if (treat_year is not None and year >= treat_year) else 0
        ttt = (year - treat_year) if (treat_year is not None and year >= treat_year) else None
        treatment_data.append({
            'pow': pow_,
            'gmina': gmina_,
            'year': year,
            'treated': treated,
            'time_to_treatment': ttt
        })

df_staggered_did_gmina = pd.DataFrame(treatment_data)

# Add cumulative funding received by that year at gmina level (intensity)
df_funding_by_year_gmina = df_locations.groupby(['pow', 'gmina', 'year_start'])['funding_split'].sum().reset_index()
df_funding_by_year_gmina.columns = ['pow', 'gmina', 'year', 'funding_that_year']
df_funding_by_year_gmina['funding_cum'] = df_funding_by_year_gmina.groupby(['pow', 'gmina'])['funding_that_year'].cumsum()

df_staggered_did_gmina = df_staggered_did_gmina.merge(
    df_funding_by_year_gmina[['pow', 'gmina', 'year', 'funding_cum']],
    on=['pow', 'gmina', 'year'],
    how='left'
)
df_staggered_did_gmina['funding_cum'] = df_staggered_did_gmina['funding_cum'].fillna(0)

# Quick checks / outputs
print("Gmina-level staggered DiD (first rows):")
display(df_staggered_did_gmina.head(10))

print("Powiat-level treatment timing (sample):")
display(df_treatment_timing.sort_values('treatment_year').head(10))

Gmina-level staggered DiD (first rows):


,pow,gmina,year,treated,time_to_treatment,funding_cum
0,Biała Podlaska,Biała Podlaska,2014,0,NaN,0.000000e+00
1,Biała Podlaska,Biała Podlaska,2015,0,NaN,0.000000e+00
2,Biała Podlaska,Biała Podlaska,2016,0,NaN,0.000000e+00
3,Biała Podlaska,Biała Podlaska,2017,0,NaN,0.000000e+00
4,Biała Podlaska,Biała Podlaska,2018,0,NaN,0.000000e+00
5,Biała Podlaska,Biała Podlaska,2019,0,NaN,0.000000e+00
6,Biała Podlaska,Biała Podlaska,2020,0,NaN,0.000000e+00
7,Biała Podlaska,Biała Podlaska,2021,1,0.0,1.422843e+08
8,Biała Podlaska,Biała Podlaska,2022,1,1.0,1.840325e+08
9,Biała Podlaska,Biała Podlaska,2023,1,2.0,2.169065e+08


Powiat-level treatment timing (sample):


,pow,treatment_year,total_funding,num_projects
395,łomżyński,2014,3.480943e+08,44
41,Kraków,2014,3.375614e+09,370
283,poznański,2014,8.165890e+08,163
374,węgrowski,2014,1.057504e+08,72
270,oświęcimski,2014,6.102660e+08,152
380,zawierciański,2014,2.923150e+08,155
235,miński,2014,1.447209e+08,100
197,krakowski,2014,8.129038e+08,198
313,siedlecki,2014,9.663685e+08,102
366,wrocławski,2014,4.198985e+08,93


In [38]:
# --- AGGREGATING YOUR GMINA-LEVEL DATA TO THE POWIAT LEVEL ---

# 1. Define the aggregation rules for each column.
aggregation_rules = {
    'funding_cum': 'sum',      # Sum cumulative funding from all gminas in the powiat.
    'treated': 'max',          # If any gmina is treated (1), the powiat is treated (1).
    'time_to_treatment': 'max' # Handles NaNs and staggered start dates correctly.
}

# 2. Group your DataFrame by 'pow' and 'year', then apply the aggregation.
df_powiat_aggregated = df_staggered_did_gmina.groupby(['pow', 'year']).agg(aggregation_rules).reset_index()

# 3. (Recommended) Sort the new DataFrame for clean, ordered panel data.
df_powiat_aggregated = df_powiat_aggregated.sort_values(by=['pow', 'year'])

# 4. Display the head of the new DataFrame to confirm the result.
print(df_powiat_aggregated.head(10))


              pow  year   funding_cum  treated  time_to_treatment
0  Biała Podlaska  2014  0.000000e+00        0                NaN
1  Biała Podlaska  2015  0.000000e+00        0                NaN
2  Biała Podlaska  2016  0.000000e+00        0                NaN
3  Biała Podlaska  2017  0.000000e+00        0                NaN
4  Biała Podlaska  2018  0.000000e+00        0                NaN
5  Biała Podlaska  2019  0.000000e+00        0                NaN
6  Biała Podlaska  2020  0.000000e+00        0                NaN
7  Biała Podlaska  2021  1.422843e+08        1                0.0
8  Biała Podlaska  2022  1.840325e+08        1                1.0
9  Biała Podlaska  2023  2.169065e+08        1                2.0


In [39]:
# Save outputs
df_powiat_aggregated.to_parquet(r"data\outputs\eu_flows\pol_2127_treatment_staggered_did_pow.parquet", index=False)
df_staggered_did_gmina.to_parquet(r"data\outputs\eu_flows\pol_2127_treatment_staggered_did_gmina.parquet", index=False)
df_treatment_timing_gmina.to_parquet(r"data\outputs\eu_flows\pol_2127_treatment_timing_gmina.parquet", index=False)
df_treatment_timing.to_parquet(r"data\outputs\eu_flows\pol_2127_treatment_timing_pow.parquet", index=False)

In [35]:
display(df_locations[['Contract number','Project location', 'woj', 'pow', 'funding_split']].head(10))
# display(df_locations.head(3))

,Contract number,Project location,woj,pow,funding_split
0,FAMI.01.01-IZ.00-0001/24,"WOJ: LUBELSKIE, POW: Biała Podlaska, GM: Biała...",LUBELSKIE,Biała Podlaska,16509401.81
1,FAMI.01.01-IZ.00-0001/24,"WOJ: PODLASKIE, POW: zambrowski, GM: Zambrów",PODLASKIE,zambrowski,16509401.81
2,FAMI.01.01-IZ.00-0001/25,"WOJ: MAZOWIECKIE, POW: legionowski, GM: Serock",MAZOWIECKIE,legionowski,50495285.04
3,FAMI.01.01-IZ.00-0002/24,Cały Kraj,None,None,26651952.75
4,FAMI.01.01-IZ.00-0003/24,Cały Kraj,None,None,13668617.36
5,FAMI.02.01-IZ.00-0001/24,WOJ: MAŁOPOLSKIE,MAŁOPOLSKIE,None,9634229.49
6,FAMI.02.01-IZ.00-0001/25,"WOJ: MAŁOPOLSKIE, POW: nowosądecki",MAŁOPOLSKIE,nowosądecki,900000.00
7,FAMI.02.01-IZ.00-0002/24,WOJ: ŚLĄSKIE,ŚLĄSKIE,None,9201000.00
8,FAMI.02.01-IZ.00-0003/24,WOJ: WARMIŃSKO-MAZURSKIE,WARMIŃSKO-MAZURSKIE,None,3014999.99
9,FAMI.02.01-IZ.00-0004/24,WOJ: KUJAWSKO-POMORSKIE,KUJAWSKO-POMORSKIE,None,3536976.62


#### Combine into 1 dataset

In [42]:
path_1420 = r"data/outputs/eu_flows/pol_1420_treatment_staggered_did.parquet"
path_2127 = r"data/outputs/eu_flows/pol_2127_treatment_staggered_did_pow.parquet"

df_1420 = pd.read_parquet(path_1420)
df_2127 = pd.read_parquet(path_2127)


In [53]:
df_combined = pd.concat([df_1420, df_2127], ignore_index=True)

# 1. Define the aggregation logic to resolve conflicts.
aggregation_rules = {
    'treated': 'max',           # max(1, 0) -> 1
    'funding_cum': 'sum',       # sum(2.3e8, 0.0) -> 2.3e8
    'time_to_treatment': 'max'  # max(4.0, NaN) -> 4.0
}

df_corrected = df_combined.groupby(['pow', 'year']).agg(aggregation_rules).reset_index()


In [55]:
df_corrected.sort_values(by=['pow', 'year']).head(5)

,pow,year,treated,funding_cum,time_to_treatment
0,Biała Podlaska,2014,1,2.646463e+07,0.0
1,Biała Podlaska,2015,1,3.447804e+07,1.0
2,Biała Podlaska,2016,1,1.091886e+08,2.0
3,Biała Podlaska,2017,1,1.469402e+08,3.0
4,Biała Podlaska,2018,1,2.116581e+08,4.0


In [ ]:
(df_corrected.treated == 0)

56

In [ ]:
# Save combined dataset
df_combined.to_parquet(r"data\clean\eu_flows\pol_combined_1420_2127_panel.parquet", index=False)